В пятилетнем рандомизированном исследовании Гарвардской медицинской школы 11037 испытуемых через 
день принимали аспирин, а ещё 11034 — плацебо 
Исследование было слепым, то есть, испытуемые не знали, что именно они принимают.
За 5 лет инфаркт случился у 104 испытуемых, принимавших аспирин, и у 189 принимавших плацебо.


In [9]:
import numpy as np
import pandas as pd
from scipy.stats import norm
%pylab inline

Populating the interactive namespace from numpy and matplotlib


/usr/local/lib/python3.7/dist-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['norm']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


Оцените, насколько вероятность инфаркта снижается при приёме аспирина. Округлите ответ до четырёх знаков после десятичной точки.

In [16]:
n_placebo = 11034
n_attack_placebo = 189
p_placebo = n_attack_placebo / n_placebo
n_aspirin = 11037
n_attack_aspirin = 104
p_aspirin = n_attack_aspirin / n_aspirin
print(p_placebo - p_aspirin)

0.0077060239760047815


Постройте теперь 95% доверительный интервал для снижения вероятности инфаркта при приёме аспирина. Чему равна его верхняя граница? Округлите ответ до четырёх знаков после десятичной точки.

In [15]:

def get_confint_by_prob(p1, p2, n1, n2, alpha=0.05):
    z = scipy.stats.norm.ppf(1 - alpha / 2.)
    delta = z * np.sqrt(p1 * (1 - p1) / n1 + p2 * (1 - p2) / n2)
    left_boundary = (p1 - p2) - delta
    right_boundary = (p1 - p2) + delta
    
    return (left_boundary, right_boundary)


get_confint_by_prob(p_placebo, p_aspirin, n_placebo, n_aspirin)

(0.004687750675049439, 0.010724297276960124)

In [22]:
def get_bootstrap_samples(data, n_samples):
    indices = np.random.randint(0, len(data), (n_samples, len(data)))
    samples = data[indices]
    return samples

In [23]:
def stat_intervals(stat, alpha):
    boundaries = np.percentile(stat, [100 * alpha / 2., 100 * (1 - alpha / 2.)])
    return boundaries

Оцените, во сколько раз понижаются шансы инфаркта при регулярном приёме аспирина. Округлите ответ до четырёх знаков после десятичной точки.

In [18]:
def get_odds(pos, total):
    p = pos / total
    return p / (1 - p)


odds_aspirin = get_odds(n_attack_aspirin, n_aspirin)
odds_placebo = get_odds(n_attack_placebo, n_placebo)
odds_placebo / odds_aspirin

Величина, которую вы оценили в предыдущем вопросе, называется отношением шансов. Постройте для отношения шансов 95% доверительный интервал с помощью бутстрепа. Чему равна его нижняя граница? Округлите ответ до 4 знаков после десятичной точки.

Чтобы получить в точности такой же доверительный интервал, как у нас:

- составьте векторы исходов в контрольной и тестовой выборках так, чтобы в начале шли все единицы, а потом все нули;
- установите random seed=0;
- сделайте по 1000 псевдовыборок из каждой группы пациентов с помощью функции get_bootstrap_samples.

In [26]:
def get_odds(data):
    p = np.sum(data) / len(data)
    return p / (1 - p)

np.random.seed(0)
n_samples = 1000
data_placebo = np.zeros(n_placebo)
data_placebo[:n_attack_placebo] = 1
data_aspirin = np.zeros(n_aspirin)
data_aspirin[:n_attack_aspirin] = 1
odds_placebo = np.fromiter(map(get_odds, get_bootstrap_samples(data_placebo, n_samples)), dtype=float)
odds_aspirin = np.fromiter(map(get_odds, get_bootstrap_samples(data_aspirin, n_samples)), dtype=float)
odds_ratio = odds_placebo / odds_aspirin
stat_intervals(odds_ratio, 0.05)

array([1.46286276, 2.35093673])